In [23]:
import numpy as np

In [24]:
def initialize():
    global T, J, JR, γ, egam, β, α, δ, tol, damp, itermax, ξ, ϵ, υ, w, r, wn, Rn, p, K, \
    L, A, C, I, B, G, k, U, Y, Ba, Bf, h, Tb, TXr, Tpen, H, eps, n_p, gy, by, κ, lsra_on, \
    smopec, τk,  τc, τw, τr, τp, τs, tax, pen, a, e, c, v, m


    T = 27; J = 3; JR = 3; γ = .5; egam = 1.0 - 1.0/γ; β = .9
    α = .3; δ = .0; tol = 1e-6; damp = 0.3
    itermax = 200; ξ = 2.0; ϵ = 0.0; υ = 0.5


    w  = np.zeros(T+1); r = np.zeros(T+1); wn = np.zeros(T+1)
    Rn = np.zeros(T+1); p = np.zeros(T+1); K  = np.zeros(T+1)
    L  = np.zeros(T+1); A = np.zeros(T+1); C  = np.zeros(T+1)
    I  = np.zeros(T+1); B = np.zeros(T+1); G  = np.zeros(T+1)
    k  = np.zeros(T+1); U = np.zeros((J,T+1)); Y = np.zeros(T+1)
    Ba = np.zeros(T+1); Bf = np.zeros(T+1); h = np.zeros((J,T+1))
    Tb = np.zeros(T+1); TXr = np.zeros(T+1); Tpen = np.zeros(T+1)
    H = np.ones(T+1); eps = np.zeros(T+1)


    n_p = np.zeros(T+1) + .2    
    gy  = 0.205
    by  = np.zeros(T+1)
    κ   = np.zeros(T+1)
    lsra_on = False
    smopec = False


    τk  = np.zeros(T+1)
    τc = np.zeros(T+1)
    τw = np.zeros(T+1)
    τr = np.zeros(T+1)
    τp = np.zeros(T+1)
    τs = np.zeros(T+1)
    tax = np.ones(T+1)
    pen = np.zeros((J,T+1))


    a = np.zeros((J,T+1))
    e = np.zeros(T+1)
    c = np.zeros((J,T+1))
    v = np.zeros(T+J-1)
    h[0,:] = 1.0
    h[1:J,:] = .0


    m = np.zeros((J,T+1))
    for t in range(T+1):
        m[0,t] = 1.0
        tm = year(t, 1, 0)
        for j in range(1,J):
            m[j,t] = m[j-1,tm]/(1.0 + n_p[t])
    return None

In [25]:
def year(t, j, jp):
    year = t + jp - j
    if t == 0 or year<=0:
        year = 0
    if t == T or year>=T:
        year = T
    return year

In [26]:
def factor_prices(t):
    global k, r, w, wn, Rn, p
    k[t]   = K[t]/L[t]                
    if smopec and t > 0:
        r[t] = r[0]
    else:
        r[t] = (1.0 - τk[t])/(1.0 - eps[t]*τk[t])*(α*k[t]**(α-1.0)*H[t]**ϵ-δ)
  
    w[t]   = (1.0 - α)*k[t]**α*H[t]**ϵ

    wn[t]  = w[t]*(1.0 - τw[t] - τp[t])    
    Rn[t]  = 1.0 + r[t]*(1.0 - τr[t])  
    p[t]   = 1.0 + τc[t]         
    return None

In [27]:
def get_W(j, t):
    
    Assets = wn[t]*h[j,t] + pen[j,t]
    
    if t == 1 and j > 0:
        Assets = Assets + Rn[t] * a[j,t] + v[J - j - 1] ## calful about index!!!
    if j == 0:
        Assets = Assets + v[t+J-2] - (1.0 -τs[t])*e[t]*h[0,t]*wn[t]
        
    PRn = 1.0
    for jp in range(j+1,J):
        tp = year(t,j,jp)
        PRn = PRn*Rn[tp]
        Assets = Assets + (wn[tp]*h[jp,tp] + pen[jp,tp])/PRn
    return Assets


def get_Psi(j,t):
    Psi = 1.0
    PRn = 1.0
    
    for jp in range(j+1,J):
        tp = year(t, j ,jp)
        PRn = PRn*Rn[tp]
        Psi = Psi + β**((jp-j)*γ)*(p[tp]/(PRn*p[t]))**(1.0-γ)
    Psi = 1.0/(p[t]*Psi)
    
    return Psi

In [28]:
def get_path(j,t):
    
    global c, a
    
    PRn = 1.0

    for jp in range(j+1,J):
        
        tp = year(t,j,jp)
        tm = year(t,j,jp-1)
        PRn = PRn*Rn[tp]
        

        c[jp,tp] = (β**(jp-j)*PRn*p[t]/p[tp])**γ*c[j,t]
        a[jp,tp] = wn[tm]*h[jp-1,tm] + pen[jp-1,tm] + Rn[tm]*a[jp-1,tm] - p[tm]*c[jp-1,tm]
        if tp == 2:
            a[jp,tp] = a[jp,tp] + v[J-jp]
        if tp > 2 and jp == 1:
            a[jp,tp] = a[jp,tp] + v[tm+J-2]
        if jp == 1:
            a[jp,tp] = a[jp,tp] - (1.0 - τs[tm])*e[tm]*wn[tm]
    return None

In [29]:
def decisions(t):
    global c, e, h
    
    tp = year(t, 0, 1)
    e[t] = (ξ*υ*wn[tp]/(wn[t]*(1.0-τs[t])*Rn[tp]))**(1.0/(1.0-υ))
    h[1,tp] = 1.0 + ξ*e[t]**υ
    
    c[0,t] = get_Psi(0,t)*get_W(0,t)
    get_path(0,t)
    
    if t == 1:
        for j in range(1,J):
            c[j,t] = get_Psi(j,t)*get_W(j,t)
            get_path(j,t)
    return None

In [30]:
## calculating quantities in a certain year

def quantities(t):
    global G, C, A, L, H, Y, B, K, Bf, Tb, I
    
    tm = year(t,1,0)
    tp = year(t,0,1)
    
    if t == 0:
        G[t] = gy*Y[t]
    else:
        G[t] = G[0]
        
    # aggregate individual decisions
    C[t] = .0
    A[t] = .0
    L[t] = .0
    H[t] = .0
    
    for j in range(J):
        C[t] = C[t] + c[j,t]*m[j,t]
        A[t] = A[t] + a[j,t]*m[j,t]
        L[t] = L[t] + h[j,t]*m[j,t]
        if j < JR-1:
            H[t] = H[t] + m[j,t]
            
    L[t] = L[t] - e[t]*h[0,t]
    H[t] = L[t]/(H[t] - e[t])
    
    Y[t] = K[t]**α*L[t]**(1.0-α)*H[t]**ϵ
    B[t] = by[tm]*Y[t]
    
    # derive capital in small open or closed economy
    if smopec and t>0:
        K[t]  = L[t]*((r[t]*(1.0-eps[t]*τk[t])/(1.0-τk[t]) + δ)/(α*H[t]**ϵ))**(1.0/(α-1.0)) 
        Bf[t] = A[t] - K[t] - Ba[t] - B[t]
        Tb[t] = (1.0+n_p[tp])*Bf[tp] - (1.0+r[t])*Bf[t]
    else:
        K[t] = damp*(A[t]-B[t]-Ba[t]) + (1.0-damp)*K[t]
    
    I[t] = (1.0+n_p[tp])*K[tp] - (1.0 - δ)*K[t]
    
    return None

In [31]:
# calculating government parameters

def government(t):
    global τc, τk, τw, τr, pen, Tpen, τp, TXr
    tp = year(t, 0, 1)
    
    taxrev = np.zeros(5)
    
    taxrev[0] = τc[t]*C[t]
    taxrev[1] = τw[t]*w[t]*L[t]
    taxrev[2] = τr[t]*r[t]*A[t]
    taxrev[3] = τk[t]*(Y[t] - w[t]*L[t]-(δ+eps[t]*r[t])*K[t])
    taxrev[4] = τs[t]*e[t]*wn[t]
    
   
    
    # get budget balancing tax rate
    if tax[t] == 1:      #(government surplus-τcC)/C
        τc[t] = (taxrev[4] + (1.0 + r[t])*B[t] + G[t] - (taxrev[1] + taxrev[2] + \
                    taxrev[3]+(1.0 + n_p[tp])*B[tp]))/C[t]   
    elif tax[t] == 2:    #[government surplus-τw(wL)-τr(rA)]/(wL+rA) assuming same labour and capital income tax
        τw[t] = (taxrev[4] + (1.0 + r[t])*B[t] + G[t] - (taxrev[0] + taxrev[3] + \
                    (1.0+n_p[tp])*B[tp]))/(w[t]*L[t] + r[t]*A[t])
        τr[t] = τw[t]    #same labour and capital income tax
    elif tax[t] == 3:    #[government surplus-τw(WL)]/(wL)
        τw[t] = (taxrev[4] + (1.0 + r[t])*B[t] + G[t] - (taxrev[0] + taxrev[2] + \
                    taxrev[3] + (1.0 + n_p[tp])*B[tp]))/(w[t]*L[t])
    else:                #[government surplus-τr(rA)]/(rA)
        τr[t] = (taxrev[4] + (1.0 + r[t])*B[t] + G[t] - (taxrev[0] + taxrev[1] + \
                    taxrev[3]+ (1.0 + n_p[tp])*B[tp]))/(r[t]*A[t])
    
    TXr[t] = np.sum(taxrev[0:4]) - taxrev[4]
    
    # get budget balancing social security contribution
    pen[JR-1:J,t] = κ[t]*w[t]
    Tpen[t] = .0
    
    for j in range(JR-1,J):
        Tpen[t] = Tpen[t] + pen[j,t]*m[j,t]
        
    τp[t] = Tpen[t]/(w[t]*L[t])
    return None

In [32]:
## Compute household utility

def utility(t):
    global U
    
    # for first generation
    U[0,t] = .0
    for j in range(J):
        tp = year(t, 0, j)
        U[0,t] = U[0,t] + β**j*c[j,tp]**egam/egam
    
    # for current total population if year = 0 or 1
    if t < 2:
        for j in range(1,J):
            U[j,t] = .0
            for jp in range(j,J):
                tp = year(t, j, jp)
                U[j,t] = U[j,t] + β**(jp-j)*c[jp,tp]**egam/egam
                
    return None

In [33]:
## Calculating lsra transfers

def lsra():
    global Ba, v, ustar

    for t in range(1,T+1):
        utility(t)   
        
    Ba = np.zeros(T+1)
    for j in range(1,J):
        v[J-j-1] = v[J-j-1] + get_W(j,1)*((U[j,0]/U[j,1])**(1.0/egam)-1.0)
        Ba[2] = Ba[2] + v[J-j-1]*m[j,1]
    
    PV = v[T+J-2]*(1.0+r[T])/(r[T]-n_p[T])
    sum1 = get_W(0,T)*(1.0+r[T])/(r[T]-n_p[T])
    sum2 = get_W(0,T)*(U[0,T]*egam)**(-1.0/egam)*(1.0+r[T])/(r[T]-n_p[T])
    
    for t in range(T-1,0,-1):
        tp = year(t,0,1)
        PV = PV*(1.0+n_p[tp])/(1.0+r[tp]) + v[t+J-2]
        sum1 = sum1*(1.0+n_p[tp])/(1.0+r[tp]) + get_W(0,t)
        sum2 = sum2*(1.0+n_p[tp])/(1.0+r[tp]) + get_W(0,t)*(U[0,t]*egam)**(-1.0/egam)
        
    ustar = ((sum1 - Ba[2] - PV)/sum2)**egam/egam
    
    for t in range(1,T+1):
        v[t+J-2] = v[t+J-2] + get_W(0,t)*((ustar/U[0,t])**(1.0/egam)-1.0)
        if t == 2:
            Ba[2] = (Ba[2] + v[J-1])/(1.0 + n_p[2])
        if t > 2:
            Ba[t] = ((1.0+r[t-1])*Ba[t-1] + v[J+t-3])/(1.0+n_p[t])
    
    return None

In [34]:
def get_SteadyState():
    K[0] = 1.0
    L[0] = 1.0
    for i in range(itermax+1):
        factor_prices(0)
        decisions(0)
        quantities(0)
        government(0)
        if abs(Y[0]-C[0]-I[0]-G[0])/Y[0] < tol:
            break
    utility(0)
    
    if i < itermax:
        print('Iteration: ', i, ' Diff: ',abs(Y[0]-C[0]-I[0]-G[0])/Y[0])
    else:
        print('!!! No equilibrium found !!!')
    
    return None

In [35]:
## sovles for transition path using Gauss-Seidel:

def get_Transition():
    global a, K, L, HEV, H, h, Δ
    # Initialize values from initial equilibrium
    a[:,1] = a[:,0]
    h[1,1] = h[1,0]
    K[:] = K[0]
    L[:] = L[0]
    H[:] = H[0]
    
    for ite in range(itermax+2):
        # get prices, decisions and quantites
        for t in range(T+1):
            factor_prices(t)
        
        for t in range(T+1):
            decisions(t)
        
        if lsra_on:
            lsra()
        
        for t in range(T+1):
            quantities(t)
        
        for t in range(T+1):
            government(t)
            
        # check for the number of markets in equilibrium
        nmarket = 0
        for t in range(1,T+1):
            if  abs(Y[t]-C[t]-I[t]-G[t]-Tb[t])/Y[t] < tol:
                nmarket = nmarket + 1
        if nmarket == T:
            break
    if ite < itermax:
        print('Iteration: ',ite,' Markets: ',nmarket,' Diff: ',np.max(np.abs(Y-C-I-G-Tb)/Y))
    else:
        print('!!! No equilibrium found !!!')
    if lsra_on == True:
        Δ = (ustar/U[0,0])**(1/egam)-1
    
    for t in range(T+1):
        utility(t)
    HEV = np.zeros(T+2)
    for t in range(2,T+2):
        HEV[t] = ((U[0,t-1]/U[0,0])**(1/egam)-1)*100
    HEV[1] = ((U[1,1]/U[1,0])**(1/egam)-1)*100 # generation 0
    HEV[0] = ((U[2,1]/U[2,0])**(1/egam)-1)*100 # generation -1
    return None

In [36]:
## Table 7.4
initialize()
tax[1:] = 3 # Table 7.4
get_SteadyState()
get_Transition()
print('---------------------------------------------')
print('Table 7.4 From consumption to labour-income \ntaxation with variable labour supply')
print('---------------------------------------------')
print(' t   τw   τc    C    e    h2   L    K    w    r    HEV')
print(-1,'                                            ','%2.2f'%HEV[0])
list1 = [0, 1, 2, 3, 4]
for t in list1:
    print('%2.f'%t,'%.2f'%τw[t],'%.2f'%τc[t],'%.2f'%C[t],'%.2f'%e[t],'%.2f'%h[1,t],'%.2f'%L[t],'%.2f'%K[t],'%.2f'%w[t],'%.2f'%r[t],'%2.2f'%HEV[t+1])
print(' .   .    .    .    .    .    .    .    .    .    . ')
print(' .   .    .    .    .    .    .    .    .    .    . ')
print(' .   .    .    .    .    .    .    .    .    .    . ')
for t in [T]:
    print(' T','%.2f'%τw[t],'%.2f'%τc[t],'%.2f'%C[t],'%.2f'%e[t],'%.2f'%h[1,t],'%.2f'%L[t],'%.2f'%K[t],'%.2f'%w[t],'%.2f'%r[t],'%2.2f'%HEV[t+1])
print('---------------------------------------------')
initialize()
tax[1:] = 3 # Table 7.4
get_SteadyState()
lsra_on = True
get_Transition()
print(Δ)

Iteration:  44  Diff:  9.598363679568825e-07
Iteration:  57  Markets:  27  Diff:  8.743007465655273e-07
---------------------------------------------
Table 7.4 From consumption to labour-income 
taxation with variable labour supply
---------------------------------------------
 t   τw   τc    C    e    h2   L    K    w    r    HEV
-1                                              27.96
 0 0.00 0.27 0.82 0.13 1.72 2.31 0.18 0.33 1.76 -2.66
 1 0.29 0.00 0.88 0.09 1.72 2.35 0.18 0.33 1.79 -10.85
 2 0.32 0.00 0.75 0.09 1.60 2.24 0.15 0.31 2.03 -16.93
 3 0.33 0.00 0.72 0.09 1.60 2.25 0.13 0.30 2.19 -19.75
 4 0.34 0.00 0.70 0.09 1.59 2.24 0.12 0.29 2.29 -21.55
 .   .    .    .    .    .    .    .    .    .    . 
 .   .    .    .    .    .    .    .    .    .    . 
 .   .    .    .    .    .    .    .    .    .    . 
 T 0.35 0.00 0.67 0.08 1.58 2.23 0.11 0.29 2.44 -24.04
---------------------------------------------
Iteration:  44  Diff:  9.598363679568825e-07
Iteration:  61  Markets:  27  Diff

In [37]:
## Table 7.5
initialize()
tax[2:] = 3 # Table 7.5
get_SteadyState()
get_Transition()
print('---------------------------------------------')
print('Table 7.5 From consumption to labour-income \ntaxation with policy announcement')
print('---------------------------------------------')
print(' t   τw   τc    C    e    h2   L    K    w    r    HEV')
print(-1,'                                            ','%2.2f'%HEV[0])
list1 = [0, 1, 2, 3, 4]
for t in list1:
    print('%2.f'%t,'%.2f'%τw[t],'%.2f'%τc[t],'%.2f'%C[t],'%.2f'%e[t],'%.2f'%h[1,t],'%.2f'%L[t],'%.2f'%K[t],'%.2f'%w[t],'%.2f'%r[t],'%2.2f'%HEV[t+1])
print(' .   .    .    .    .    .    .    .    .    .    . ')
print(' .   .    .    .    .    .    .    .    .    .    . ')
print(' .   .    .    .    .    .    .    .    .    .    . ')
for t in [T]:
    print(' T','%.2f'%τw[t],'%.2f'%τc[t],'%.2f'%C[t],'%.2f'%e[t],'%.2f'%h[1,t],'%.2f'%L[t],'%.2f'%K[t],'%.2f'%w[t],'%.2f'%r[t],'%2.2f'%HEV[t+1])
print('---------------------------------------------')
initialize()
tax[2:] = 3 # Table 7.5
lsra_on = True
get_SteadyState()
get_Transition()
print(HEV[T+1])

Iteration:  44  Diff:  9.598363679568825e-07
Iteration:  57  Markets:  27  Diff:  8.743007465655273e-07
---------------------------------------------
Table 7.5 From consumption to labour-income 
taxation with policy announcement
---------------------------------------------
 t   τw   τc    C    e    h2   L    K    w    r    HEV
-1                                              1.04
 0 0.00 0.27 0.82 0.13 1.72 2.31 0.18 0.33 1.76 6.32
 1 0.00 0.27 0.82 0.08 1.72 2.36 0.18 0.33 1.79 -0.52
 2 0.30 0.00 0.86 0.10 1.56 2.20 0.20 0.34 1.63 -8.51
 3 0.31 0.00 0.79 0.09 1.62 2.26 0.16 0.31 1.94 -14.95
 4 0.33 0.00 0.73 0.09 1.60 2.24 0.14 0.30 2.13 -18.83
 .   .    .    .    .    .    .    .    .    .    . 
 .   .    .    .    .    .    .    .    .    .    . 
 .   .    .    .    .    .    .    .    .    .    . 
 T 0.35 0.00 0.67 0.08 1.58 2.23 0.11 0.29 2.44 -24.04
---------------------------------------------
Iteration:  44  Diff:  9.598363679568825e-07
Iteration:  33  Markets:  27  Diff:  0.08

In [38]:
## Table 7.6
initialize()
τs[1:] = .25 # Table 7.6
get_SteadyState()
get_Transition()
print('---------------------------------------------')
print('Table 7.6 Education subsidies without human-capital externalities')
print('---------------------------------------------')
print(' t   τs   τc    C    e   h2    L    K    w    r   HEV')
print(-1,'                                            ','%2.2f'%HEV[0])
list1 = [0, 1, 2, 3, 4]
for t in list1:
    print('%2.f'%t,'%.2f'%τs[t],'%.2f'%τc[t],'%.2f'%C[t],'%.2f'%e[t],'%.2f'%h[1,t],'%.2f'%L[t],'%.2f'%K[t],'%.2f'%w[t],'%.2f'%r[t],'%2.2f'%HEV[t+1])
print(' .   .    .    .    .    .    .    .    .    .    . ')
print(' .   .    .    .    .    .    .    .    .    .    . ')
print(' .   .    .    .    .    .    .    .    .    .    . ')
for t in [T]:
    print(' T','%.2f'%τs[t],'%.2f'%τc[t],'%.2f'%C[t],'%.2f'%e[t],'%.2f'%h[1,t],'%.2f'%L[t],'%.2f'%K[t],'%.2f'%w[t],'%.2f'%r[t],'%2.2f'%HEV[t+1])
print('---------------------------------------------')
initialize()
τs[1:] = .25 # Table 7.6
lsra_on = True
get_SteadyState()
get_Transition()
print(HEV[T+1])

Iteration:  44  Diff:  9.598363679568825e-07
Iteration:  30  Markets:  27  Diff:  9.712278054163521e-07
---------------------------------------------
Table 7.6 Education subsidies without human-capital externalities
---------------------------------------------
 t   τs   τc    C    e   h2    L    K    w    r   HEV
-1                                              -3.30
 0 0.00 0.27 0.82 0.13 1.72 2.31 0.18 0.33 1.76 0.23
 1 0.25 0.29 0.81 0.20 1.72 2.23 0.18 0.33 1.73 1.50
 2 0.25 0.29 0.83 0.21 1.90 2.37 0.17 0.32 1.86 -0.16
 3 0.25 0.29 0.83 0.21 1.92 2.39 0.17 0.32 1.88 -0.49
 4 0.25 0.29 0.83 0.21 1.92 2.39 0.17 0.32 1.90 -0.85
 .   .    .    .    .    .    .    .    .    .    . 
 .   .    .    .    .    .    .    .    .    .    . 
 .   .    .    .    .    .    .    .    .    .    . 
 T 0.25 0.29 0.82 0.21 1.92 2.39 0.17 0.32 1.91 -1.12
---------------------------------------------
Iteration:  44  Diff:  9.598363679568825e-07
Iteration:  45  Markets:  27  Diff:  0.010203613411862196


In [39]:
## Table 7.7
initialize()
ϵ = .2;τs[1:] = .25 # Table 7.7
get_SteadyState()
get_Transition()
print('---------------------------------------------')
print('Table 7.7 Education subsidies with human-capital externalities')
print('---------------------------------------------')
print(' t   τs   τc    C    e   h2    L    K    w    r   HEV')
print(-1,'                                            ','%2.2f'%HEV[0])
list1 = [0, 1, 2, 3, 4]
for t in list1:
    print('%2.f'%t,'%.2f'%τs[t],'%.2f'%τc[t],'%.2f'%C[t],'%.2f'%e[t],'%.2f'%h[1,t],'%.2f'%L[t],'%.2f'%K[t],'%.2f'%w[t],'%.2f'%r[t],'%2.2f'%HEV[t+1])
print(' .   .    .    .    .    .    .    .    .    .    . ')
print(' .   .    .    .    .    .    .    .    .    .    . ')
print(' .   .    .    .    .    .    .    .    .    .    . ')
for t in [T]:
    print(' T','%.2f'%τs[t],'%.2f'%τc[t],'%.2f'%C[t],'%.2f'%e[t],'%.2f'%h[1,t],'%.2f'%L[t],'%.2f'%K[t],'%.2f'%w[t],'%.2f'%r[t],'%2.2f'%HEV[t+1])
print('---------------------------------------------')
initialize()
lsra_on = True
ϵ = .2;τs[1:] = .25 # Table 7.7
get_SteadyState()
get_Transition()
print(Δ)

Iteration:  46  Diff:  8.795355329609461e-07
Iteration:  31  Markets:  27  Diff:  7.786120085053085e-07
---------------------------------------------
Table 7.7 Education subsidies with human-capital externalities
---------------------------------------------
 t   τs   τc    C    e   h2    L    K    w    r   HEV
-1                                              -3.04
 0 0.00 0.27 0.90 0.13 1.72 2.31 0.20 0.36 1.76 1.08
 1 0.25 0.29 0.89 0.20 1.72 2.23 0.20 0.36 1.73 2.62
 2 0.25 0.28 0.92 0.21 1.90 2.37 0.19 0.35 1.90 1.90
 3 0.25 0.28 0.93 0.21 1.92 2.39 0.19 0.35 1.91 2.03
 4 0.25 0.28 0.93 0.21 1.92 2.39 0.19 0.35 1.91 1.91
 .   .    .    .    .    .    .    .    .    .    . 
 .   .    .    .    .    .    .    .    .    .    . 
 .   .    .    .    .    .    .    .    .    .    . 
 T 0.25 0.28 0.93 0.21 1.92 2.39 0.19 0.35 1.91 1.85
---------------------------------------------
Iteration:  46  Diff:  8.795355329609461e-07
Iteration:  45  Markets:  27  Diff:  0.023258321224896236
-0.0003